## 1 Data Importing and Initial Cleaning

We start our data preprocessing by loading the datasets. The `steam_data` contains information about the games available on the Steam platform, and `user_data` includes user interactions with these games.

### 1.1 Loading the Datasets

- The Steam games dataset is loaded from a CSV file into a DataFrame called `steam_data`.
- The user data is also loaded from a CSV file into a DataFrame called `user_data`. We provide custom column names for clarity and consistency.

### 1.2 Initial Data Cleaning

- In the `user_data` DataFrame, we immediately drop a column named 'To drop', which is not needed for our analysis.
- By displaying the first few rows of the user data, we can quickly check if the data is loaded correctly and the unwanted column is removed.


In [16]:
import numpy as np
import pandas as pd

# Load the Steam games data
steam_data = pd.read_csv('../data/steam.csv')

# Define new column names for the user data
column_names = ['User_ID', 'Game_Name', 'Action', 'Hours', 'To drop']

# Load the user data with the specified column names
user_data = pd.read_csv('../data/user.csv', header=None, names=column_names)

# Drop the unnecessary 'To drop' column from the user data
user_data.drop('To drop', axis=1, inplace=True)

# Display the first few rows of the user data to verify
user_data.head()

,User_ID,Game_Name,Action,Hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [17]:
steam_data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


## 2 Text Cleaning and Data Consistency

To ensure consistency between our datasets, we need to clean the game titles and filter our data so that we only consider the intersection of games present in both `steam_data` and `user_data`.

### 2.1 Cleaning Game Titles

- We define a function `clean_text` that takes a string and removes punctuation and unnecessary whitespace. It also converts the text to lowercase.
- This cleaning function is applied to the 'name' column in `games_df` and the 'Game_Name' column in `filtered_user_df` to standardize the game titles across both datasets.

In [18]:
import re

# Function to clean game titles by removing punctuation and extra spaces
def clean_text(text):
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuations
    text = text.lower().strip()          # Convert to lowercase and strip extra spaces
    return text

# Create copies of the original DataFrames to work with
games_df = steam_data.copy()
filtered_user_df = user_data.copy()

# Apply the cleaning function to game titles in both datasets
games_df['cleaned_title'] = games_df['name'].apply(clean_text)
filtered_user_df['cleaned_game_title'] = filtered_user_df['Game_Name'].apply(clean_text)

### 2.2 Filtering Data for Consistency

- We filter `filtered_user_df` to keep only the rows where the cleaned game titles exist in `games_df`. This ensures that we only consider user interactions with games that we have data for in our Steam games dataset.
- Similarly, we filter `games_df` to include only those games that are present in the cleaned user interactions data.

In [19]:
# Filter user data to include only those games present in the steam data
filtered_user_df_cleaned = filtered_user_df[filtered_user_df['cleaned_game_title'].isin(games_df['cleaned_title'])]

# Filter the games data to include only those games that are present in the user data
games_df = games_df[games_df['cleaned_title'].isin(filtered_user_df_cleaned['cleaned_game_title'])]

## 3 Checking for Missing Values and Duplicates

An essential part of data preprocessing is ensuring the integrity of our data by checking for and handling missing values and duplicate entries. These issues can affect the quality of our analysis and the performance of our models.

### 3.1 Missing Values

- We use the `.isnull().sum()` function to count the number of missing values in each column of both the `games_df` and `filtered_user_df_cleaned` DataFrames.
- It is important to identify if there are any missing values that could indicate problems with data collection or errors in previous data processing steps.

### 3.2 Duplicates

- Duplicated rows can skew our analysis, so we use the `.duplicated().sum()` function to count the number of duplicate rows in our datasets.
- Identifying duplicates is crucial because it can reveal data entry errors or issues with how data is collected or merged from different sources.

### 3.3 Handling Missing Values and Duplicates

- If missing values are found, we need to decide whether to fill them with appropriate values or to drop the rows or columns containing them, depending on the extent and nature of the missing data.
- In the case of duplicates, after confirming that they are indeed unintended repetitions and not just similar entries, we can remove them using the `.drop_duplicates()` method to maintain the uniqueness of each record.

In [20]:
# Check for missing values in the games data
missing_values_games = games_df.isnull().sum()
# Check for missing values in the user data
missing_values_users = filtered_user_df_cleaned.isnull().sum()

# Display missing values information
print("Missing values in games data:\n", missing_values_games)
print("\nMissing values in user data:\n", missing_values_users)

# Check for duplicate rows in the games data
duplicate_rows_games = games_df.duplicated().sum()
# Check for duplicate rows in the user data
duplicate_rows_users = filtered_user_df_cleaned.duplicated().sum()

# Display duplicate row information
print("\nDuplicate rows in games data:", duplicate_rows_games)
print("Duplicate rows in user data:", duplicate_rows_users)

# If there are duplicates,  remove them
games_df = games_df.drop_duplicates()
filtered_user_df_cleaned = filtered_user_df_cleaned.drop_duplicates()

Missing values in games data:
 appid               0
name                0
release_date        0
english             0
developer           0
publisher           0
platforms           0
required_age        0
categories          0
genres              0
steamspy_tags       0
achievements        0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
owners              0
price               0
cleaned_title       0
dtype: int64

Missing values in user data:
 User_ID               0
Game_Name             0
Action                0
Hours                 0
cleaned_game_title    0
dtype: int64

Duplicate rows in games data: 0
Duplicate rows in user data: 91


## 4 Saving the Preprocessed Data

- After filtering, the cleaned and consistent datasets are saved back to CSV files. This step ensures that we have a set of data that is ready for analysis and modeling, with all game titles standardized and unnecessary records removed.
- Saving the preprocessed data at this stage allows us to easily share the cleaned datasets with team members or use them in different stages of the project without repeating the preprocessing steps.

In [14]:
# Save the filtered datasets to new CSV files
games_df.to_csv('../data/games_filtered.csv', index=False)
filtered_user_df_cleaned.to_csv('../data/users_filtered.csv', index=False)

**For recommendation system, please refer to *'3-Recommendation System.ipynb'***   